<a href="https://colab.research.google.com/github/sauravsingla/Multi-Class-Sentiment-Classification-/blob/main/twitter_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive') 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pip install bert-for-tf2

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow_hub as hub
from bert import bert_tokenization
BertTokenizer = bert_tokenization.FullTokenizer

In [ ]:
import pandas as pd
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input, Flatten
from tensorflow.keras.layers import Bidirectional, GRU, GlobalMaxPool1D, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
import time
import nltk
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import KernelPCA
from pandas import DataFrame
import re
import numpy as np

In [ ]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "what is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"don't", "do not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"didn't", "did not", text)
    return text

In [ ]:
df = pd.read_csv("/content/drive/My Drive/NLP/text_emotion.csv")
df = df.drop(["tweet_id","author"],axis = 1)
print(df.head())

    sentiment                                            content
0       empty  @tiffanylue i know  i was listenin to bad habi...
1     sadness  Layin n bed with a headache  ughhhh...waitin o...
2     sadness                Funeral ceremony...gloomy friday...
3  enthusiasm               wants to hang out with friends SOON!
4     neutral  @dannycastillo We want to trade with someone w...


In [ ]:
df.drop(df[df['sentiment'] == 'empty'].index, inplace = True)
df.drop(df[df['sentiment'] == 'enthusiasm'].index, inplace = True)
df['target'] = df['sentiment'].map({'sadness':0, 'boredom':1,'neutral':2,'worry':3,'surprise':4,'love':5,'fun':6,'hate':7,'happiness':8,'anger':9,'relief':10})
df = df.drop(["sentiment"],axis=1)
print(df.head())

                                             content  target
1  Layin n bed with a headache  ughhhh...waitin o...       0
2                Funeral ceremony...gloomy friday...       0
4  @dannycastillo We want to trade with someone w...       2
5  Re-pinging @ghostridah14: why didn't you go to...       3
6  I should be sleep, but im not! thinking about ...       0


In [ ]:
cleaned_text = []
for text in df['content']:
    cleaned_text.append(clean_text(text))
df['clean'] = cleaned_text
df.head()

,content,target,clean
1,Layin n bed with a headache ughhhh...waitin o...,0,layin n bed with a headache ughhhh...waitin o...
2,Funeral ceremony...gloomy friday...,0,funeral ceremony...gloomy friday...
4,@dannycastillo We want to trade with someone w...,2,@dannycastillo we want to trade with someone w...
5,Re-pinging @ghostridah14: why didn't you go to...,3,re-pinging @ghostridah14: why did not you go t...
6,"I should be sleep, but im not! thinking about ...",0,"i should be sleep, but im not! thinking about ..."


In [ ]:
tw = []
for j in df['clean']:
  tweets = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", str(j)).split())
  tw.append(tweets)
df['clean'] = tw
df.head()

,content,target,clean
1,Layin n bed with a headache ughhhh...waitin o...,0,layin n bed with a headache ughhhh waitin on y...
2,Funeral ceremony...gloomy friday...,0,funeral ceremony gloomy friday
4,@dannycastillo We want to trade with someone w...,2,we want to trade with someone who has houston ...
5,Re-pinging @ghostridah14: why didn't you go to...,3,re pinging why did not you go to prom bc my bf...
6,"I should be sleep, but im not! thinking about ...",0,i should be sleep but im not thinking about an...


In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
stopwords = set(stopwords.words('english'))
output = []
for sentence in df["clean"]:
    temp_list = []
    for word in sentence.split():
      if len(word) > 2 not in stopwords:
        temp_list.append(word)
    output.append(' '.join(temp_list))
    
df["texts"] = output
df = df.drop(['content','clean'], axis = 1)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()

w_tokenizer = nltk.tokenize.WhitespaceTokenizer()

def lemmatize_text(texts):
    return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(texts)]


df['text_lemmatized'] = df.texts.apply(lemmatize_text)

sc = [[' '.join(i)] for i in df['text_lemmatized']]
lis = []
for i in sc:
    abc = i[0]
    lis.append(abc)

df['lem'] = lis
df = df.drop(['text_lemmatized','texts'], axis =1)
df.head()

,target,lem
1,0,layin bed with headache ughhhh waitin your call
2,0,funeral ceremony gloomy friday
4,2,want trade with someone who ha houston ticket ...
5,3,pinging why did not you prom did not like friend
6,0,should sleep but not thinking about old friend...


In [ ]:
from sklearn.model_selection import train_test_split
from keras.utils import np_utils
X = df["lem"]
Y = df["target"]
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X,Y, test_size = 0.30)
Yt = np_utils.to_categorical(Ytrain)
Yt

array([[0., 0., 0., ..., 1., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [ ]:
dft=pd.DataFrame(data=Yt[0:,0:],index=[i for i in range(Yt.shape[0])],columns=['f'+str(i) for i in range(Yt.shape[1])])
dft.head()

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
module_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1"
bert_layer = hub.KerasLayer(module_url, trainable=True)

In [ ]:
def bert_encode(texts, tokenizer, max_len=40):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)      
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [ ]:
def build_model(bert_layer, max_len=44):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")
    _, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    clf_output = sequence_output[:, 0, :]
    out = Dense(11, activation='softmax')(clf_output)
    model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    model.compile(Adam(lr=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

    return model

In [ ]:
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = bert_tokenization.FullTokenizer(vocab_file, do_lower_case)

In [ ]:
# train_input = bert_encode(df..values, tokenizer, max_len=20)
# test_input = bert_encode(df2.content.values, tokenizer, max_len=20)
# train_labels = df1.target.values
train_input = bert_encode(Xtrain, tokenizer, max_len=40)
test_input = bert_encode(Xtest, tokenizer, max_len=40)
train_labels = dft

In [ ]:
model = build_model(bert_layer, max_len=40)
model.summary()

Model: "functional_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 40)]         0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 40)]         0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 40)]         0                                            
__________________________________________________________________________________________________
keras_layer_2 (KerasLayer)      [(None, 1024), (None 335141889   input_word_ids[0][0]             
                                                                 input_mask[0][0]      

In [ ]:
train_history = model.fit(
    train_input, train_labels,
    epochs=3,
    batch_size=32,
    validation_split=0.4
)

Epoch 1/3
505/505 [==============================] - 522s 1s/step - loss: 1.8025 - accuracy: 0.3563 - val_loss: 1.7002 - val_accuracy: 0.3944
Epoch 2/3
414/505 [=======================>......] - ETA: 1:16 - loss: 1.5584 - accuracy: 0.4482

In [ ]:
test_pred = model.predict(test_input)


In [ ]:
import matplotlib.pyplot as plt
plt.plot(train_history.history['loss'], label='loss')
plt.plot(train_history.history['val_loss'], label='val_loss')
plt.legend()
plt.show()

# accuracies
plt.plot(train_history.history['accuracy'], label='acc')
plt.plot(train_history.history['val_accuracy'], label='val_acc')
plt.legend()
plt.show()

In [ ]:
p = np.argmax(test_pred, axis=1)
print(p)

from sklearn.metrics import confusion_matrix
cnfv=confusion_matrix(Ytest,p)
print(cnfv)

from sklearn.metrics import classification_report
print(classification_report(Ytest, p))